<a href="https://colab.research.google.com/github/1712832wine/VietNameseQuestionLabels/blob/main/VietNameseQuestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Data

In [53]:
import csv
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
train = {
    "sentences": [],
    "labels": []
}
test = {
    "sentences": [],
    "labels": []
}

In [54]:
def read_data(filepath):
    with open(filepath, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            if (row[1] != ''):
                if (row[4]=='TRAIN'):
                    train['sentences'].append(row[1])
                    train['labels'].append(row[2])
                else:
                    test['sentences'].append(row[1])
                    test['labels'].append(row[2])

In [55]:
#10 câu hỏi đầu tiên
read_data("input.csv")
print(train['sentences'][:10])
print(test['sentences'][:10])

['Con gà là con gì', 'Định nghĩa định lí pytago là gì', 'Thế nào gọi là bò sát', 'Thế nào gọi là gia cầm', 'Thế nào gọi là động vật có vú', 'Thế nào gọi là hiệu ứng nhà kính', 'Định nghĩa của môi trường sống', 'Quả cam có vị chua như thế nào', 'Trời hôm nay nóng như thế nào', 'Cơm mẹ nấu ngon như thế nào']
['Định nghĩa của thuyết tiến hóa', 'Thế nào gọi là sự nóng lên toàn cầu', 'Thế nào gọi là xe máy', 'Con gà trông như thế nào', 'Quả chanh chua như thế nào', 'Con chó sủa như thế nào', 'Con mèo kêu như thế nào', 'Tại sao nên ăn chín uống sôi', 'Tại sao nên đi ngủ sớm', 'Màu đỏ là màu gì']


In [74]:
# các loại nhãn trong tập train
set_labels = Counter(train['labels']).keys()
print(set_labels)
# labels to id
label2id = {x: i for i, x in enumerate(set_labels)}
print(label2id)

dict_keys(['DESC.def', 'DESC.desc', 'DESC.reason', 'ENTY.language', 'ENTY.sport', 'HUM.desc', 'HUM.group', 'HUM.ind', 'LOC.city', 'LOC.country', 'NUM.data', 'ABBR.abb', 'ABBR.exp', 'ENTY', 'LOC', 'NUM', 'NUM.date', 'SELECT', 'ENTY.animal', 'ENTY.plant', 'ENTY.religion', 'Select', 'ENT'])
{'DESC.def': 0, 'DESC.desc': 1, 'DESC.reason': 2, 'ENTY.language': 3, 'ENTY.sport': 4, 'HUM.desc': 5, 'HUM.group': 6, 'HUM.ind': 7, 'LOC.city': 8, 'LOC.country': 9, 'NUM.data': 10, 'ABBR.abb': 11, 'ABBR.exp': 12, 'ENTY': 13, 'LOC': 14, 'NUM': 15, 'NUM.date': 16, 'SELECT': 17, 'ENTY.animal': 18, 'ENTY.plant': 19, 'ENTY.religion': 20, 'Select': 21, 'ENT': 22}


In [57]:
# độ lớn tập train và tập test
print("#training size", len(train['sentences']))
print("#testing size", len(test['sentences']))

#training size 1837
#testing size 442


# Sentences/Labels to Vector


In [58]:
# chuyển nhãn từ chữ thành số dựa vào id 
train_target = [label2id[x] for x in train['labels']]
print(train_target)
print(len(train_target))

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 14, 14, 14, 14, 14, 14, 14, 14, 14, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 13, 13, 13, 13, 13, 13, 13, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 14, 14, 14, 14, 14, 14, 14, 14, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 15, 15, 15, 15, 15, 1

In [59]:
count_vect = CountVectorizer(ngram_range=(1,2))
X_train_counts = count_vect.fit_transform(train['sentences'])
print(X_train_counts.shape)
print(X_train_counts.toarray())

(1837, 8813)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [60]:
# method 1: no-transofrm
X_train = X_train_counts
# method 2: use tf
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train = tf_transformer.transform(X_train_counts)
# method 3: use tf-idf
# tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
# X_train = tf_transformer.transform(X_train_counts)
# print(X_train.shape)
print(X_train.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [61]:
# method 1
# MultinomialNB
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(X_train, train_target)
# method 2
# from sklearn.linear_model import SGDClassifier
# clf = SGDClassifier(loss='hinge', penalty='l2',
# alpha=1e-3, random_state=42,
# max_iter=100, tol=None)
# clf.fit(X_train, train_target)
# method 3
clf = svm.LinearSVC()
clf.fit(X_train, train_target)
# method 4
#clf = svm.SVC(kernel='linear')
# # clf = svm.SVC(kernel='rbf')
#clf.fit(X_train, train_target)
# method 5
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(penalty='l2', verbose=1)
# clf.fit(X_train, train_target)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

# Predict

In [63]:
#id to labels
id2label = {i: x for i, x in enumerate(set_labels)}

In [66]:
X_new_counts = count_vect.transform(test['sentences'])
X_new = X_new_counts
# X_new = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new)
for doc, category in zip(test['sentences'], predicted):
  print(doc, category,":", id2label[category])

Định nghĩa của thuyết tiến hóa 0 : DESC.def
Thế nào gọi là sự nóng lên toàn cầu 0 : DESC.def
Thế nào gọi là xe máy 0 : DESC.def
Con gà trông như thế nào 1 : DESC.desc
Quả chanh chua như thế nào 1 : DESC.desc
Con chó sủa như thế nào 1 : DESC.desc
Con mèo kêu như thế nào 1 : DESC.desc
Tại sao nên ăn chín uống sôi 2 : DESC.reason
Tại sao nên đi ngủ sớm 2 : DESC.reason
Màu đỏ là màu gì 13 : ENTY
Diện tích sân bóng đá là bao nhiêu 15 : NUM
Cô ấy có hiếu không 5 : HUM.desc
Cô ta có học giỏi không 5 : HUM.desc
Ngoại hình cô ấy như thế nào 5 : HUM.desc
Anh ta và cô ta quen nhau bao lâu rồi 7 : HUM.ind
Sinh viên trường tự nhiên có giỏi không 17 : SELECT
Bạn thích uống cafe nào 7 : HUM.ind
Hôm nay ngày bao nhiêu 16 : NUM.date
 Ngày ra đời của  máy đếm tiền 10 : NUM.data
viết tắt của trung tâm kiểm soát và phòng ngừa dịch bệnh là gì? 11 : ABBR.abb
cm là viết tắt của gì? 12 : ABBR.exp
Nguyệt thực là gì? 0 : DESC.def
Thuỷ triều là gì? 0 : DESC.def
trí tuệ nhân tạo là gì 0 : DESC.def
nghịch lý tiền 

# Evaluation

In [75]:
# các loại nhãn trong tập test
set_label_test = Counter(test['labels']).keys()
# labels to id
label2id_test = {x: i for i, x in enumerate(set_label_test)}
# chuyển nhãn từ chữ thành số dựa vào id 
test_target = [label2id_test[x] for x in test['labels']]
print(test_target)
print(len(test_target))

[0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, 8, 9, 10, 0, 0, 0, 0, 0, 1, 1, 2, 2, 5, 6, 6, 6, 6, 6, 7, 7, 7, 11, 12, 13, 13, 13, 13, 14, 14, 9, 9, 9, 9, 9, 10, 10, 10, 1, 15, 15, 15, 5, 6, 6, 7, 7, 11, 11, 12, 12, 12, 13, 13, 13, 16, 16, 17, 17, 17, 9, 9, 9, 10, 10, 0, 0, 0, 0, 1, 1, 2, 2, 2, 15, 5, 6, 6, 6, 7, 7, 11, 11, 12, 12, 13, 13, 13, 14, 16, 16, 0, 1, 2, 2, 2, 15, 15, 15, 5, 5, 5, 6, 6, 7, 7, 7, 7, 7, 11, 11, 12, 12, 12, 12, 13, 13, 13, 14, 16, 16, 17, 17, 17, 17, 17, 9, 9, 10, 10, 0, 1, 1, 1, 1, 2, 2, 5, 5, 6, 6, 7, 7, 7, 12, 12, 13, 16, 17, 17, 17, 17, 18, 18, 19, 20, 9, 9, 10, 0, 0, 0, 1, 1, 1, 15, 15, 5, 6, 6, 6, 7, 11, 11, 12, 12, 13, 13, 14, 14, 14, 16, 16, 16, 17, 9, 9, 10, 0, 0, 1, 1, 1, 2, 2, 15, 6, 7, 7, 7, 7, 7, 11, 11, 11, 12, 12, 12, 13, 14, 16, 16, 16, 17, 17, 17, 9, 9, 9, 9, 10, 0, 1, 1, 2, 15, 15, 15, 5, 6, 7, 7, 11, 12, 12, 12, 13, 13, 14, 14, 17, 14, 15, 15, 15, 21, 21, 21, 13, 12, 2, 2, 0, 0, 6, 6, 11, 11, 7, 7, 5, 10, 10, 9, 9, 9, 9, 9, 10, 0, 0, 0

In [79]:
X_new_counts = count_vect.transform(test['sentences'])
X_new = X_new_counts
# X_new = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new)
print(len(predicted))
print("predicted")
print(predicted)
print("Gold Label")
print(test_target)

442
predicted
[ 0  0  0  1  1  1  1  2  2 13 15  5  5  5  7 17  7 16 10 11 12  0  0  0
  0  0  1  1  2  2  5 16  6  6  6 13  7  5  7 14  8  9  9  9 14 15 16 11
 11 11 11 11 12 12 12  1  1  0  9  5  5  6  7  7 14 14  8  8  8  9  9  9
 16 16  7 15  0 12  0  0 17  0  0 12  1  0  1 14  6  0  1 13  1  6  6  6
  7  7 14  8 14  8  9  9  9 15 16 16  0  1  2  2  2 13 13 13  5  5  5  6
  6  6  6  7  7  5 14  8  8  8  8  8  9  9  9 15 16 16 17 17 17 17 17 11
 11  3 12  0  1  1  7  0  2  2  5  5  6  6  7  7  7  8  8  9 16 17 17 17
 17  9 18  2 20 11 11 12  0  0  0  1 13  0 13 13  5  6  6  6  6  9  9  8
  8  9  9 15 15 15 16 16 16 17 11 11 11  0  0 22  9  1  2  2 13 14  7  7
  7  7  6 14 14 14  8  8  8  9 16 16 16 16 15 17 17 11 11 11 12 12  0  0
  1  2 13 13  0  5  6  7  7 14  8  8  8 14  9 15 15 17 15 13 13 13 17 17
 17  9  8 16  2  0  0  6  0 14  0  7  7  5 12 12 11 11 11 12 11 12  0  0
  0  0 12 12  5  7  7 13 14 14 14 14 14 14  9  9 15 15 16 16 16 16 11 11
 12 12 12  0  0  0  0 12  5  5  6  9 

In [81]:
ncorrect = sum([y_pred == y for y_pred, y in zip (predicted, test_target)])
print("correc:",ncorrect)
print("total:",len(test_target))
accurracy = ncorrect / len(test_target)
print("accurracy", accurracy)

correc: 195
total: 442
accurracy 0.4411764705882353
